# Check Training & Validation Set

In [40]:
import os

training_data_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_data_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

print(f" Training Subfolders: {os.listdir(training_data_path)}")
print(f" Validation Subfolders: {os.listdir(validation_data_path)}")

 Training Subfolders: ['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']
 Validation Subfolders: ['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']


# Split The Dataset Into Training & Validation

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (48, 48)
BATCH_SIZE = 64

training_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

validation_data_generator = ImageDataGenerator(rescale=1./255)

training_generator = training_data_generator.flow_from_directory(
   training_data_path,
   color_mode="grayscale",
   target_size=IMAGE_SIZE,
   batch_size=BATCH_SIZE,
   class_mode="categorical",
   shuffle=True
)

validation_generator = validation_data_generator.flow_from_directory(
   validation_data_path,
   color_mode="grayscale",
   target_size=IMAGE_SIZE,
   batch_size=BATCH_SIZE,
   class_mode="categorical",
   shuffle=True
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


# Print The Class Indices(angry, disgust, fear, happy etc.) Which We Need Later For Inferencing

In [42]:
print(training_generator.class_indices)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


# Create The Facial Emotion Recognition CNN (Convolutional Neural Network)

In [44]:
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

EMOTIONS_COUNT = 7

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=(48, 48, 1)))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(EMOTIONS_COUNT, activation="softmax"))

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)                   │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 44, 44, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 20, 20, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │       2,097,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,489,095 (9.50 MB)

 Trainable params: 2,489,095 (9.50 MB)

 Non-trainable params: 0 (0.00 B)

# Train The Facial Emotion Recognition CNN By 30 Epochs

In [45]:
EPOCHS = 30
history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

Epoch 1/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 189s 407ms/step - accuracy: 0.2503 - loss: 1.7994 - val_accuracy: 0.3279 - val_loss: 1.6795
Epoch 2/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.3287 - loss: 1.6791 - val_accuracy: 0.4244 - val_loss: 1.4835
Epoch 3/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.4038 - loss: 1.5332 - val_accuracy: 0.4925 - val_loss: 1.3433
Epoch 4/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.4500 - loss: 1.4281 - val_accuracy: 0.5167 - val_loss: 1.2723
Epoch 5/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 40s 90ms/step - accuracy: 0.4684 - loss: 1.3811 - val_accuracy: 0.5429 - val_loss: 1.1958
Epoch 6/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.4982 - loss: 1.3189 - val_accuracy: 0.5432 - val_loss: 1.1999
Epoch 7/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 44s 97ms/step - accuracy: 0.5103 - loss: 1.2860 - val_accuracy: 0.5709 - val_loss: 1.1426
Epoch 8/30
451/451 ━━━━━━━━━━━━━━━━━━━━ 42s 93ms/step - accuracy: 0.5200 - loss: 1.2588 

# Print The Model Loss & Accuracy Matrix

In [46]:
loss, accuracy = model.evaluate(validation_generator)
print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

111/111 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.6254 - loss: 1.0141
Training Loss: 1.0214083194732666
Training Accuracy: 0.6197282671928406


# Save The Trained Model In Kaggle Working Directory

In [47]:
model.save("/kaggle/working/facial_emotion_recognition.h5")

# Download The Saved Model From Kaggle

In [48]:
from IPython.display import FileLink

FileLink("facial_emotion_recognition.h5") 

/kaggle/working/facial_emotion_recognition.h5